<a href="https://www.kaggle.com/code/yonkotoshiro/sentimentanalysis?scriptVersionId=174467838" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Семантический анализ распознавания эмоций
Данный датасет представляет собой большой набор текста для обучения распознавания эмоций. Всего имеется два столбца - текст и соответствующую ему эмоция.
Мы попробуем построить несколько несложных моделей, которые смогут проанализировать текст на наличие заданной эмоции и выявлять её, и проанализируем их качество.

### Загрузка и обработка данных
В целом, данные довольно хорошие и не требуют большой обработки, но кое-что сделать все же будет необходимо.

In [ ]:
import pandas as pd
df = pd.read_csv('/kaggle/input/emotion-analysis-based-on-text/emotion_sentimen_dataset.csv', index_col = 0)
df = df.rename(columns={'Emotion': 'emotions'})
print(df.shape)
df.head()

In [ ]:
df.emotions.value_counts()

Как мы видим, есть довольно сильный дисбаланс классов. К сожалению, особо с этим ничего не поделаешь (не хотелось бы делать даунсэмплинг).
Тем не менее, для скорости обучения и простоты уменьшим количество нейтральных эмоций до 40000 и уберем последнюю эмоций с очень маленьким количеством записей.


In [ ]:
df[df.emotions == 'neutral'] = df[df.emotions == 'neutral'].sample(40000) # берем 40к случайных записей
df = df[df.emotions != 'boredom'] # удаляем данную эмоцию
df = df.dropna()
df.emotions.value_counts()

Сделаем небольшую выборку, на которой можно более подробно посмотреть текст для обучения.

In [ ]:
df_test = df.copy().sample(30, random_state = 73) # берем 30 случайных записей
for i in df_test.text:
    print(i)

Далее проведем обработку текста - приведем его к нижнему регистру,удалим специальные символы и цифры, а затем выделим токены и леммы, используя SpaCy.

In [ ]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import string

nlp = spacy.load("en_core_web_sm") # загружаем модуль для английского языка

import string

In [ ]:
def preprocess_text(text):
    # Приведение текста к нижнему регистру
    text = text.lower()
    # Удаление лишних символов 
    text = ''.join([char for char in text if char not in string.punctuation and char not in '’—‘”“' and char.isalpha() or char == ' '])
    # Удаление чисел
    text = ''.join([char for char in text if not char.isdigit()])
    # Инициализация объекта для обработки текста
    doc = nlp(text)
    # Токенизация текста и удаление стоп-слов
    tokens = [token.text for token in doc if token.text not in STOP_WORDS]
    # Лемматизация токенов
    lemmatized_tokens = [token.lemma_ for token in doc if token.lemma_ not in STOP_WORDS]
    # Объединение обработанных слов
    lemmatized_tokens = ' '.join(lemmatized_tokens)
    # Возвращение предобработанного текста
    return lemmatized_tokens

# Примененение функции к тестовому датафрейму
df_test['preprocessed_text'] = df_test['text'].apply(preprocess_text) 
df_test.head()

После этого применяем функцию ко всему датафрейму (займет много времени, потому его сохраняем)

In [ ]:
# Примененение функции к основному датафрейму
df['preprocessed_text'] = df['text'].apply(preprocess_text) 
df.head()

# Сохранение нового датафрейма
df.to_csv('processed_text.csv')

In [ ]:
df.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['encoded_emotions'] = label_encoder.fit_transform(df.emotions)

## Модели

### Логистическая регрессия
Для начала используем самую простую модель - логистическую регрессию.

Делим датафрейм на тренировочные данные и признаки.

In [ ]:
X = df.preprocessed_text
y = df.emotions

Делаем векторизацию текста.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(min_df=0, max_df=0.95, binary=False, ngram_range=(1,3))
tf_train = tf.fit_transform(X)
tf_test = tf.fit_transform(y)
print('Bow_tf_train',tf_train.shape)
print('Bow_tf_test',tf_test.shape)

Создаеем саму модель

In [ ]:
%%time

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer

# Создаем объект CountVectorizer
vectorizer = CountVectorizer()

# Конвертируем текст
X_bow = vectorizer.fit_transform(df.text)

# Делим данные на тренировочные и тестовые
X_train, X_test, y_train, y_test = train_test_split(X_bow, df.emotions, test_size=0.2, random_state=73)

# Инициализируем модель логистической регрессии
logistic = LogisticRegression(penalty='l2', max_iter=500, C=1, random_state=73)

# Подставляем данные
lr_bow = logistic.fit(X_train, y_train)

# Делаем предсказания на тестовых данных
y_pred = lr_bow.predict(X_test)

# Рассчитываем точность
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Выдаем Classification Report
print("Classification Report:")
print(classification_report(y_test, y_pred))

In [ ]:
examples = {
    'anger': "I can't believe she said that to me! How dare she!",
    'empty': "I feel like there's nothing left inside me. I'm just numb.",
    'enthusiasm': "I can't wait to start my new project! I'm so excited about it!",
    'fun': "Last night's party was so much fun! We danced all night long!",
    'happiness': "I just got promoted at work! I'm really happy!",
    'hate': "I can't stand that guy. Every time he speaks, I just feel hatred.",
    'love': "Being with you makes me feel so loved and cherished.",
    'neutral': "I don't really have strong feelings about this. It's just okay.",
    'relief': "I finally finished my exams! What a relief!",
    'sadness': "Losing my pet was one of the saddest moments of my life.",
    'surprise': "I can't believe you remembered my birthday! What a pleasant surprise!",
    'sorry': "I can't stop thinking about the future. What if things don't work out?"}

# Обработка текста 
processed_examples = {emotion: preprocess_text(text) for emotion, text in examples.items()} 
# Векторизация текста
example_features = vectorizer.transform(processed_examples.values())
# Предсказание эмоций для каждого примера
predicted_emotions = lr_bow.predict(example_features)
# Подсчет правильных предсказаний
correct_predictions = sum(1 for predicted_emotion, true_emotion in zip(predicted_emotions, examples.keys()) if predicted_emotion == true_emotion)

print(f"Количество правильных предсказаний: {correct_predictions}/{len(examples)}\n")

for i, (emotion, text) in enumerate(processed_examples.items()):
    print(f"Текст: {text}, предсказанная эмоция: {predicted_emotions[i]} ({emotion})")


### CatBoost

In [ ]:
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_bow, df.emotions, test_size=0.2, random_state=73)

In [ ]:
%%time

# Создание и обучение модели 
model = CatBoostClassifier(iterations=500, learning_rate=0.1, depth=6, loss_function='MultiClass', random_seed=73)
model.fit(X_train, y_train, verbose = 100)

y_pred = model.predict(X_test)

# Оценка производительности модели
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(y_test, y_pred))

In [ ]:
examples = {
    'anger': "I can't believe she said that to me! How dare she!",
    'empty': "I feel like there's nothing left inside me. I'm just numb.",
    'enthusiasm': "I can't wait to start my new project! I'm so excited about it!",
    'fun': "Last night's party was so much fun! We danced all night long!",
    'happiness': "I just got promoted at work! I'm really happy!",
    'hate': "I can't stand that guy. Every time he speaks, I just feel hatred.",
    'love': "Being with you makes me feel so loved and cherished.",
    'neutral': "I don't really have strong feelings about this. It's just okay.",
    'relief': "I finally finished my exams! What a relief!",
    'sadness': "Losing my pet was one of the saddest moments of my life.",
    'surprise': "I can't believe you remembered my birthday! What a pleasant surprise!",
    'sorry': "I can't stop thinking about the future. What if things don't work out?"}

# Обработка текста 
processed_examples = {emotion: preprocess_text(text) for emotion, text in examples.items()} 
# Векторизация текста
example_features = vectorizer.transform(processed_examples.values())
# Предсказание эмоций для каждого примера
predicted_emotions = model.predict(example_features)

print(f"Количество правильных предсказаний: {correct_predictions}/{len(examples)}\n")

for i, (emotion, text) in enumerate(processed_examples.items()):
    print(f"Текст: {text}, предсказанная эмоция: {predicted_emotions[i]} ({emotion})")
